In [1]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import notebook_login
from datasets import load_dataset
torch.cuda.empty_cache()

## 1. 허깅페이스 로그인

In [2]:
notebook_login()

## 2. 데이터셋 로드

In [3]:
# TEST_NAME = "beomi-gemma-ko-2b-01"
# TEST_NAME = "google-gemma-ko-2b-01"
# TEST_NAME = "beomi-gemma-ko-7b"
# TEST_NAME = "beomi-gemma-mling-7b"
TEST_NAME = "upstage-SOLAR-10.7B-Instruct-v1.0"

import os
import platform

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
TRAIN_PATH = DATA_PATH + "train_kkh_new.csv"
VALID_PATH = DATA_PATH + "dev_kkh_new.csv"
TEST_PATH = DATA_PATH + "test.csv"
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
SUBMIT_PATH = PREDICTION_PATH + "submission_" + TEST_NAME + ".csv"

# BASE_MODEL = "google/gemma-2b-it"
# BASE_MODEL = "beomi/gemma-ko-2b"
# BASE_MODEL = "beomi/gemma-ko-7b"
# BASE_MODEL = "beomi/gemma-mling-7b"
BASE_MODEL = "upstage/SOLAR-10.7B-Instruct-v1.0"
FINETUNE_MODEL = PRE_PATH + f"gemma_model/{TEST_NAME}"

DATASET_TRAIN = load_dataset('csv', data_files={'train': TRAIN_PATH})
DATASET_TEST = load_dataset('csv', data_files={'test': TEST_PATH})
TRAIN_DATA = DATASET_TRAIN['train']

In [4]:
# 데이터셋 확인
print(TRAIN_DATA[0])
print('=============================')
print(f"train 대화문:\n{TRAIN_DATA[0]['dialogue']}")
print('=============================')
print(f"train 요약문:\n{TRAIN_DATA[0]['summary']}")
print('=============================')
print(f"test 대화문:\n{TRAIN_DATA[0]['dialogue']}")

{'fname': 'train_0', 'dialogue': '#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?\n#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.\n#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.\n#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?\n#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.\n#Person2#: 알겠습니다.\n#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?\n#Person2#: 네.\n#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. \n#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.\n#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.\n#Person2#: 알겠습니다, 감사합니다, 의사선생님.', 'summary': '스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.', 'topic': '건강검진 받기'}
train 대화문:
#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 

## 3.1 모델 로드

In [5]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=quantization_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
# ex = TRAIN_DATA[0]['dialogue']
# messages = [
#     {
#         "role": "user",
#         "content": "다음 글을 해설자가 설명하듯이 요약해주세요.:\n\n{}".format(ex)
#     }
# ]
# prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# prompt

In [7]:
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
# outputs = pipe(
#     prompt,
#     do_sample=True,
#     temperature=0.5,
#     top_k=50,
#     top_p=0.95
# )

In [8]:
# print(outputs[0]["generated_text"][len(prompt):])

In [9]:
ex = TRAIN_DATA[0]['dialogue']

conversation = [ {'role': 'user', 'content': "다음 글을 해설자가 설명하듯이 요약해주세요.:\n\n{}".format(ex)} ] 

prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device) 
outputs = model.generate(**inputs, use_cache=True, max_length=1024)
output_text = tokenizer.decode(outputs[0]) 
print(output_text)

/opt/conda/envs/nlp/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<s> ### User:
다음 글을 해설자가 설명하듯이 요약해주세요.:

#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.

### Assistant:
In the given text, Person 1, a doctor from Hockins, greets Person 2 and inquires about their visit. Person 2 mentions they came for a health check-up and the doctor points out that it has been 5 years since their last check-up and it should ideally be done annually. Person 2 acknowledges this but questions the need for regular check-ups if they have no apparent hea

In [10]:
ex = TRAIN_DATA[0]['dialogue']

conversation = [ {'role': 'user', 'content': "다음 글을 해설자가 설명하듯이 한국어로 요약해주세요.:\n\n{}".format(ex)} ] 

prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device) 
outputs = model.generate(**inputs, use_cache=True, max_length=1024)
output_text = tokenizer.decode(outputs[0]) 
print(output_text)

<s> ### User:
다음 글을 해설자가 설명하듯이 한국어로 요약해주세요.:

#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.

### Assistant:
이글은 #Person1 (Dr. Hockins, a doctor)과 #Person2 (Mr. Smith)의dialogue. #Person1은 Mr.Smith를안부와함겐Mr.Smith의 recentvisit에대해explains. Mr.Smith는regularhealthcheck를받기위함과5년의interval을mentions. Dr.Hockins는earlydiseasedetection과preventativemeasures의importance를highlights. TheydiscussMr.Smith's eye and ear health,andthenmoveon tothetopicof smoking. Mr.Smithadmits tobeing a s

## 4.1 학습용 프롬프트 조정

In [11]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['dialogue'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}

<end_of_turn>
<start_of_turn>model

{}

<end_of_turn><eos>""".format(example['dialogue'][i], example['summary'][i]))
    return prompt_list

In [12]:
prompt_ex_one = generate_prompt(TRAIN_DATA[:1])[0]
print(prompt_ex_one)

<bos><start_of_turn>user
다음 글을 요약해주세요:

#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요?
#Person2#: 건강검진을 받는 것이 좋을 것 같아서요.
#Person1#: 그렇군요, 당신은 5년 동안 건강검진을 받지 않았습니다. 매년 받아야 합니다.
#Person2#: 알고 있습니다. 하지만 아무 문제가 없다면 왜 의사를 만나러 가야 하나요?
#Person1#: 심각한 질병을 피하는 가장 좋은 방법은 이를 조기에 발견하는 것입니다. 그러니 당신의 건강을 위해 최소한 매년 한 번은 오세요.
#Person2#: 알겠습니다.
#Person1#: 여기 보세요. 당신의 눈과 귀는 괜찮아 보입니다. 깊게 숨을 들이쉬세요. 스미스씨, 담배 피우시나요?
#Person2#: 네.
#Person1#: 당신도 알다시피, 담배는 폐암과 심장병의 주요 원인입니다. 정말로 끊으셔야 합니다. 
#Person2#: 수백 번 시도했지만, 습관을 버리는 것이 어렵습니다.
#Person1#: 우리는 도움이 될 수 있는 수업과 약물들을 제공하고 있습니다. 나가기 전에 더 많은 정보를 드리겠습니다.
#Person2#: 알겠습니다, 감사합니다, 의사선생님.

<end_of_turn>
<start_of_turn>model

스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니다. 호킨스 의사는 스미스씨가 담배를 끊는 데 도움이 될 수 있는 수업과 약물에 대한 정보를 제공할 것입니다.

<end_of_turn><eos>


## 4.2 QLoRA 설정

In [13]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [14]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## 4.3 Trainer 실행

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=TRAIN_DATA,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        max_steps=100,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_ratio=0.02,
        learning_rate=2e-4,
        fp16=True,
        fp16_full_eval=True,  # kkh 추가함
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/12448 [00:00<?, ? examples/s]

/opt/conda/envs/nlp/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
max_steps is given, it will override any value given in num_train_epochs


In [16]:
trainer.train()

Step,Training Loss
100,0.912200


TrainOutput(global_step=100, training_loss=0.9122486877441406, metrics={'train_runtime': 336.9033, 'train_samples_per_second': 1.187, 'train_steps_per_second': 0.297, 'total_flos': 1.163460422172672e+16, 'train_loss': 0.9122486877441406, 'epoch': 0.032133676092544985})

## 4.4 Finetuned Model 저장

In [17]:
ADAPTER_MODEL = "lora_adapter"
trainer.model.save_pretrained(ADAPTER_MODEL)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained(FINETUNE_MODEL)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 

## 5.1 Fine-tuned 모델 로드

In [13]:
finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB. GPU 

## 5.2 Fine-tuned 모델 추론

In [18]:
# ex = DATASET_TEST['test']['dialogue'][0]
ex = DATASET_TRAIN['train']['dialogue'][0]
messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요:\n\n{}".format(ex)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [19]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

#Person2#는 스미스씨에게 건강검진을 받고 싶다고 말합니다. #Person1#은 스미스씨에게 도움이 될 수 있는 정보와 약물을 제공하고 있습니다.


